In [13]:
import pandas as pd

In [14]:
#Теперь поработаем с данными о поездках на такси в Нью-Йорке, в которых также имеется информация о погодных условиях и выходных днях.  
taxi = pd.read_csv('/content/2_taxi_nyc.csv', encoding='windows-1251', sep=',')

In [15]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


In [16]:
#Проверьте, сколько всего строк и столбцов имеется в датасете.
taxi.shape

(29101, 14)

In [17]:
#Давайте посмотрим на типы колонок. Все ли из них считались правильно? В качестве ответа выберите тип, преобладающий в датасете.
taxi.dtypes

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object

In [18]:
#В названиях столбцов pcp 01, pcp 06, pcp 24 встречается пробел. Это не очень удобно, если в дальнейшем вы планируете обращаться к колонкам через точку, без использования кавычек и скобочек. 
#Замените пробел в названиях на знак нижнего подчеркивания. 
taxi.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp 01', 'pcp 06', 'pcp 24', 'sd'],
      dtype='object')

In [19]:
taxi = taxi.rename (columns ={'pcp 01': 'pcp_01', 
                              'pcp 06': 'pcp_06', 
                              'pcp 24': 'pcp_24'})

In [20]:
#Вас попросили узнать, сколько записей (строк) в датафрейме относятся к району Манхэттен (Manhattan)
taxi.query("borough == 'Manhattan'").shape[0]

4343

In [21]:
#А если мы хотим посмотреть, сколько раз встречается каждый из районов? 

taxi['borough'].value_counts()

Manhattan        4343
Staten Island    4343
Queens           4343
Bronx            4343
Brooklyn         4343
EWR              4343
Name: borough, dtype: int64

In [22]:
#Для начала, посчитайте общее количество поездок (pickups), без группировки.

taxi.pickups.sum()

14265773

In [23]:
taxi.pickups.mean()

490.2159032335659

In [24]:
#выяснить, из какого района было совершено наибольшее количество поездок за весь период. 

taxi.groupby(['borough'], as_index = False) \
  .aggregate({'pickups':'sum'}) \
  .sort_values('pickups', ascending = False)

,borough,pickups
3,Manhattan,10367841
1,Brooklyn,2321035
4,Queens,1343528
0,Bronx,220047
5,Staten Island,6957
2,EWR,105


In [25]:
#Сохраните название района с наименьшим числом поездок в переменную min_pickups, применив подходящий метод

s = taxi.groupby(['borough']) \
  .aggregate({'pickups':'sum'})

min_pickups = s.pickups.idxmin()

In [28]:
#Продолжим изучение данных и посмотрим на число поездок в выходные дни. 
#Сгруппируйте данные по двум признакам: району города и является ли день выходным (колонки borough и hday). 
#Сравните среднее число поездок, и выберите районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни. 

taxi.groupby(['borough', 'hday']) \
  .aggregate({'pickups':'mean'}) 

#EWR, Queens

pickups
borough       hday             
Bronx         N       50.771073
              Y       48.065868
Brooklyn      N      534.727969
              Y      527.011976
EWR           N        0.023467
              Y        0.041916
Manhattan     N     2401.302921
              Y     2035.928144
Queens        N      308.899904
              Y      320.730539
Staten Island N        1.606082
              Y        1.497006

In [29]:
#Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные значения по убыванию и 
#сохраните результирующий датафрейм в pickups_by_mon_bor.
#Обратите внимание, что итоговый датасет должен состоять из 3-х колонок - pickup_month, borough, pickups.

pickups_by_mon_bor = taxi.groupby(['borough', 'pickup_month'], as_index = False) \
  .aggregate({'pickups':'sum'}) \
  .sort_values('pickups', ascending = False)

In [30]:
pickups_by_mon_bor

,borough,pickup_month,pickups
21,Manhattan,Jun,1995388
23,Manhattan,May,1888800
19,Manhattan,Feb,1718571
22,Manhattan,Mar,1661261
18,Manhattan,Apr,1648278
20,Manhattan,Jan,1455543
9,Brooklyn,Jun,482466
11,Brooklyn,May,476087
6,Brooklyn,Apr,378095
10,Brooklyn,Mar,346726


In [31]:
taxi.temp.max()

89.0

In [32]:
def temp_to_celcius(temp):
  return (temp - 32) * 5 / 9

In [33]:
temp_to_celcius(taxi.temp)

0        -1.111111
1        -1.111111
2        -1.111111
3        -1.111111
4        -1.111111
           ...    
29096    23.888889
29097    23.888889
29098    23.888889
29099    23.888889
29100    23.888889
Name: temp, Length: 29101, dtype: float64